In [54]:
import codecs
import numpy as np
from pprint import pprint
from numpy.linalg import norm

In [57]:
#process centroids
# infile = "/Users/curtisxuan/Desktop/Work/KNN_Users/centroids.txt"
infile = "/Users/curtisxuan/Desktop/Work/experiment2/centroidsQ.txt"
centroids = []
with codecs.open(infile, "r", "utf-8") as f:
    for line in f:
        dim = line.rstrip().split(" ")
        centroids.append(dim)
centroids = np.array (centroids,dtype='float32')
centroids = centroids[:,1:]
# print(centroids)

#parse features
featureVec = "/Users/curtisxuan/Desktop/Work/experiment2/embedding_q.txt"
count=0
features=[]
categ=[]
with codecs.open(featureVec, 'r', 'utf8') as f:
    for line in f:
        if count==0:
            count+=1
            continue
        info = line.rstrip().split('\t')
        categ.append(info[0])
        vector = [float(v) for v in info[3].split(' ')]
        features.append(vector)
rawFea = np.array (features,dtype='float32')
# normalize features
L2norm = norm(rawFea, axis=1, ord=2)
# rawFeatures=rawFeatures / L2norm.reshape(1267244,1)

rawFeatures=[]
count=0
for i in L2norm:
    if i!=0:    
        rawFeatures.append(rawFea[count]/i)
    count+=1
# print(features[0])
# print(rawFeatures[0])

# #extract ONLY CATEGORIES from features 
# rl = ['ct#poi','ct#twitter','ct#smartnews','ct#googlenews','ct#flipboard']
# categories=[]
# rawCategories=[]
# for i in features:
#     if (i[0].startswith('tct#') or (i[0].startswith('ct#') and (not '_' in i[0]) and (not (i[0].startswith(rl[0]) or i[0].startswith(rl[1]) or i[0].startswith(rl[2]) or i[0].startswith(rl[3]) or i[0].startswith(rl[4]))))):
#         categories.append(i)
# # print(len(categories))        
# for i in categories:
#     rawCategories.append(i[1:])
# rawCategories = np.array (rawCategories,dtype='float32')
# normalize category
# L2norm = norm(rawCategories, axis=1, ord=2)
# rawCategories=rawCategories / L2norm.reshape(127,1)
# print(categories[0])
# print(rawCategories[0])

In [56]:
#find top 5 features for each centroid
import requests
from lxml.html import fromstring

ct=1
minList=[]
for centroid in centroids:
    tempRaw = rawFeatures
    tempFea = categ
    top5 = []
    for iteration in range(1,6):
        count=0
        minDist=100
        minIndex=0
        for feature in tempRaw:
            dist = np.linalg.norm(centroid-feature)
        #         print(dist)
            if minDist > dist:
                minDist = dist
                minIndex = count
            count+=1
        if tempFea[minIndex].startswith('doc'):
            code = tempFea[minIndex][6:]
            r = requests.get('http://www.newsbreakapp.com/news/'+code)
            tree = fromstring(r.content)
            title = tree.findtext('.//title')
            top5.append([iteration,code +': ' + title,minDist])
            tempRaw=np.delete(tempRaw,minIndex,0)
            tempFea.pop(minIndex)
            
        else:
            top5.append([iteration,tempFea[minIndex],minDist])
            tempRaw=np.delete(tempRaw,minIndex,0)
            tempFea.pop(minIndex)
    print ([ct,top5])
    minList.append([ct,top5])
    ct+=1
pprint(minList)



[1, [[1, 'uid#11631318', 0.36224473], [2, 'uid#21714766', 0.3668763], [3, 'uid#34775746', 0.370064], [4, 'uid#33611176', 0.3746129], [5, 'uid#10576509', 0.37711737]]]
[2, [[1, 'clk#0LZWLlCz', 0.32760742], [2, 'uid#29800377', 0.3292418], [3, 'uid#31184918', 0.34204072], [4, 'clk#0LwrmeCh', 0.3462657], [5, 'uid#27695425', 0.34852204]]]
[3, [[1, 'uid#34016112', 0.29723206], [2, 'uid#14116794', 0.2997187], [3, 'uid#27882920', 0.30301583], [4, 'uid#24000069', 0.3048349], [5, 'uid#13864113', 0.30509475]]]
[4, [[1, 'uid#33595510', 0.36200118], [2, 'uid#20374154', 0.36288348], [3, 'uid#29848516', 0.37953106], [4, 'uid#18477981', 0.3872824], [5, 'uid#31200064', 0.387981]]]
[5, [[1, 'uid#33225836', 0.3902036], [2, 'clk#0MB7dzoC', 0.39045385], [3, 'uid#21164089', 0.3934913], [4, 'clk#0M5bAMme', 0.39456385], [5, 'uid#36086551', 0.40185517]]]
[6, [[1, 'dvos#12.2', 0.3626549], [2, 'op#verizon', 0.37930414], [3, 'dvn#iPhone', 0.3799184], [4, 'dvos#12.1.4', 0.40133658], [5, 'uid#23763681', 0.40352866]

[49, [[1, 'dvn#LenovoTB-8304F', 0.3081547], [2, 'clk#0M7rFzj8', 0.32465968], [3, 'clk#0MAGORZe', 0.329243], [4, 'clk#0LzC0eSc', 0.33304325], [5, 'clk#0M4Zo7Pz', 0.33794883]]]
[50, [[1, 'uid#29214970', 0.40142873], [2, 'uid#8693900', 0.40559736], [3, 'clk#0M9kepwA', 0.40868956], [4, 'uid#29573138', 0.4181062], [5, 'clk#0M74NrMa', 0.4204724]]]
[[1,
  [[1, 'uid#11631318', 0.36224473],
   [2, 'uid#21714766', 0.3668763],
   [3, 'uid#34775746', 0.370064],
   [4, 'uid#33611176', 0.3746129],
   [5, 'uid#10576509', 0.37711737]]],
 [2,
  [[1, 'clk#0LZWLlCz', 0.32760742],
   [2, 'uid#29800377', 0.3292418],
   [3, 'uid#31184918', 0.34204072],
   [4, 'clk#0LwrmeCh', 0.3462657],
   [5, 'uid#27695425', 0.34852204]]],
 [3,
  [[1, 'uid#34016112', 0.29723206],
   [2, 'uid#14116794', 0.2997187],
   [3, 'uid#27882920', 0.30301583],
   [4, 'uid#24000069', 0.3048349],
   [5, 'uid#13864113', 0.30509475]]],
 [4,
  [[1, 'uid#33595510', 0.36200118],
   [2, 'uid#20374154', 0.36288348],
   [3, 'uid#29848516', 0.3

In [44]:
print(len(categ))
len(features)

238869


238869

In [ ]:
#find top 5 categories for each centroid
ct=1
minList=[]
for centroid in centroids:
    tempRaw = rawCategories
    tempCat = categories
    top5 = []
    for iteration in range(1,6):
        count=0
        minDist=100
        minIndex=0
        for category in tempRaw:
            dist = np.linalg.norm(centroid-category)
        #         print(dist)
            if minDist > dist:
                minDist = dist
                minIndex = count
            count+=1
        top5.append([iteration,tempCat[minIndex][0],minDist])
        tempRaw=np.delete(tempRaw,minIndex,0)
        tempCat=np.delete(tempCat,minIndex,0)
    print ([ct,top5])
    minList.append([ct,top5])
    ct+=1
pprint(minList)



In [ ]:
#extract ONLY POPULAR FEATURES from features 
popFea=[]
rawPop=[]

#gather most popular features
mostPopular=set()
import json
in_file2 = "/Users/curtisxuan/Desktop/Work/KNN_Users/popularity.txt"
with open(in_file2, 'r') as file:
    popularity=json.load(file)
sorted_popularity = sorted(popularity, key=popularity.get, reverse=True)
count=0
for r in sorted_popularity:
    count+=1
    if count==5000:
        break
    mostPopular.add(r)

for i in features:
    if i[0] in mostPopular:
        popFea.append(i)
# print(len(categories))        
for i in popFea:
    rawPop.append(i[1:])
rawPop = np.array (rawPop,dtype='float32')
# normalize popular features
L2norm = norm(rawPop, axis=1, ord=2)
rawPop=rawPop / L2norm.reshape(4999,1)
# print(popFea[0])
# print(rawPop[0])
print(len(popFea))

#find top 5 popular features for each centroid
ct=1
minList=[]
for centroid in centroids:
    tempRaw = rawPop
    tempFea = popFea
    top5 = []
    for iteration in range(1,6):
        count=0
        minDist=100
        minIndex=0
        for feature in tempRaw:
            dist = np.linalg.norm(centroid-feature)
        #         print(dist)
            if minDist > dist:
                minDist = dist
                minIndex = count
            count+=1
        top5.append([iteration,tempFea[minIndex][0],minDist])
        tempRaw=np.delete(tempRaw,minIndex,0)
        tempFea=np.delete(tempFea,minIndex,0)
    print ([ct,top5])
    minList.append([ct,top5])
    ct+=1
pprint(minList)

